In [2]:
# Basic imports
import pandas as pd 
import numpy as np

In [3]:
# load the dataset
data = pd.read_csv('../Datagathering/data.csv')

In [4]:
# Display head to check the data
data.head()

,Region,Country,wind speed(m/s),Temp(C),SP(kPa),PRECTOTCORR,ASSD(kWh/m²/day),year,month
0,Americas,Dominican Republic,4.62,24.69,101.28,0.07,5.65,2000,1
1,Americas,Dominican Republic,4.08,25.28,101.18,0.08,5.83,2000,2
2,Americas,Dominican Republic,3.11,26.10,101.02,2.12,5.97,2000,3
3,Americas,Dominican Republic,2.95,27.35,100.76,0.63,6.89,2000,4
4,Americas,Dominican Republic,5.03,28.77,100.58,5.44,5.22,2000,5


In [5]:
# Display the info of the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35376 entries, 0 to 35375
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Region            35376 non-null  object 
 1   Country           35376 non-null  object 
 2   wind speed(m/s)   35376 non-null  float64
 3   Temp(C)           35376 non-null  float64
 4   SP(kPa)           35376 non-null  float64
 5   PRECTOTCORR       35376 non-null  float64
 6   ASSD(kWh/m²/day)  35376 non-null  float64
 7   year              35376 non-null  int64  
 8   month             35376 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 2.4+ MB


In [6]:
# separting num. and char. values for model training 
numeric_features = data[['year ', 'month']]
categorical_features = data[['Region', 'Country']]

In [7]:
# One-hot encoding for categorical features
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
categorical_features_encoded = ohe.fit_transform(categorical_features)
# Convert the encoded features back to a DataFrame
categorical_features_encoded = pd.DataFrame(categorical_features_encoded,columns=ohe.get_feature_names_out(categorical_features.columns))

In [8]:
#Scaling numeric features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

numeric_features_scaled = scaler.fit_transform(numeric_features)
numeric_features_scaled = pd.DataFrame(numeric_features_scaled, columns=numeric_features.columns)

In [9]:
# Concatenate numeric and encoded categorical features
processed_data = pd.concat([numeric_features_scaled, categorical_features_encoded], axis=1)

In [10]:
# Review the processed data
processed_data.head()

,year,month,Region_Americas,Region_Asia,Region_Europe,Region_Middle East and North Africa,Region_Pacific,Region_Sub-Saharan Africa,Country_Afghanistan,Country_Algeria,...,Country_Ukraine,Country_United Arab Emirates,Country_United Kingdom,Country_United States of America,Country_Uruguay,Country_Uzbekistan,Country_Venezuela,Country_Vietnam,Country_Zambia,Country_Zimbabwe
0,-1.655032,-1.593255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.655032,-1.303572,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.655032,-1.013890,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.655032,-0.724207,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.655032,-0.434524,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
x = processed_data
y = data['ASSD(kWh/m²/day)']

In [12]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
# split data into training and testing set
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Reshape input to [samples, timesteps, features]
x_train_lstm = np.expand_dims(x_train.values, axis=1)
x_test_lstm = np.expand_dims(x_test.values, axis=1)

# Build LSTM model
model = Sequential()
from tensorflow.keras import Input

model = Sequential()
model.add(Input(shape=(x_train_lstm.shape[1], x_train_lstm.shape[2])))
model.add(LSTM(64))
model.add(Dense(1))
# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(x_train_lstm, y_train, epochs=10, batch_size=64, validation_data=(x_test_lstm, y_test))

Epoch 1/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 14.1637 - val_loss: 1.9885
Epoch 2/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.8211 - val_loss: 1.2317
Epoch 3/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1.1939 - val_loss: 1.0404
Epoch 4/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.9925 - val_loss: 0.7794
Epoch 5/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.7208 - val_loss: 0.4810
Epoch 6/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4500 - val_loss: 0.3288
Epoch 7/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3117 - val_loss: 0.2481
Epoch 8/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2431 - val_loss: 0.1987
Epoch 9/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1975 - val_loss: 0.1723
Epoch 10/10
443/443 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.1711 - val_loss: 0.1558


In [14]:
y_pred = model.predict(x_test_lstm)
print('Test MSE:', mean_squared_error(y_test, y_pred))
print('Test R2:', r2_score(y_test, y_pred))

# Predict on training data
y_train_pred = model.predict(x_train_lstm)
print('Train MSE:', mean_squared_error(y_train, y_train_pred))
print('Train R2:', r2_score(y_train, y_train_pred))

222/222 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Test MSE: 0.15576325481382283
Test R2: 0.9514207968780796
885/885 ━━━━━━━━━━━━━━━━━━━━ 1s 802us/step
Train MSE: 0.15367959162139094
Train R2: 0.9528531183229241


In [15]:
# Save the model
model.save("../Models/assd_lstm_model.keras")


In [16]:
from tensorflow.keras.models import load_model
# Load the model
loaded_model = load_model("../Models/assd_lstm_model.keras")

# Function to prepare input for prediction given year, month, region, and country
def prepare_input(year, month, region, country):
    # Start with all zeros for all columns
    input_dict = {col: 0 for col in x_test.columns}
    # Scale year and month using the same mean and std as training
    input_dict['year '] = (year - numeric_features['year '].mean()) / numeric_features['year '].std()
    input_dict['month'] = (month - numeric_features['month'].mean()) / numeric_features['month'].std()
    # Set region and country one-hot columns
    region_col = f"Region_{region}"
    country_col = f"Country_{country}"
    if region_col in input_dict:
        input_dict[region_col] = 1
    if country_col in input_dict:
        input_dict[country_col] = 1
    # Create DataFrame
    return pd.DataFrame([input_dict], columns=x_test.columns)

# Example usage:
input_df = prepare_input(2025, 999, "Americas", "Dominican Republic")

# Reshape for LSTM: [samples, timesteps, features]
input_lstm = np.expand_dims(input_df.values, axis=1)

# Predict
predicted_assd = loaded_model.predict(input_lstm)
print("Predicted ASSD(kWh/m²/day):", predicted_assd[0][0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Predicted ASSD(kWh/m²/day): 2.3796554


In [17]:
import joblib

joblib.dump(ohe, '../models/encoder1.pkl')
joblib.dump(scaler, '../models/scaler1.pkl')

['../models/scaler1.pkl']